In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
data = pd.read_csv("CustFlowDataset_full_red_mcc.csv", index_col=None)
names_full = data["Unnamed: 0"].values
names_short = data["org_name"].values
categories = data[["category1", "category2", "category3"]].values
categories_mcc_string = data[["category1_mcc_string", "category2_mcc_string", 
                              "category3_mcc_string"]].values

app_keys = ["3a2eb450-eeae-4657-86e8-b531db4d912e", 
            "17047a85-f562-4c18-b8e9-a204a3093f90",
            "75f43276-fdf5-4347-ba25-9c1b4b33aec0",
            "8228fc0f-0cf3-4d11-ade8-0c79f5aa2c50", 
            "98768c74-405b-4e25-a4fa-91f1cb20af7f", 
            "74b36c35-5340-4bc3-8254-2bab716bb5ad", 
            "04019624-3bd7-45db-90a4-f61262ecc96f", 
            "996bd176-aaf5-4041-b369-933dafda0f0e", 
            "6b565a82-1988-4e04-a4bb-e36f11398db8", 
            "bd127805-3750-439b-929f-cdde13d97257", 
            "544b6025-fe5c-42c8-8b00-0ed013e67841", 
            "a97c2623-4c69-4c85-9ac0-4ace19cefdec", 
            "8c31d95e-28b3-4487-af13-239270107ce1", 
            "9bba2e35-08e7-477f-92f5-3eac5951504a", 
            "41c5632d-0935-4bc1-855f-17471937837d", 
            "f790402e-51ca-4475-88b2-2226cf47d58b"]

In [ ]:
offset = 0
step = 0

for app_key in app_keys:
    for i in range(500):
        if i + step + offset >= names_short.shape[0]:
            break
        
        request = 'https://search-maps.yandex.ru/v1/?text=' + names_full[i + step + offset] + \
            '&type=biz&ll=30.372821,59.958007&spn=0.004,0.002&lang=ru_RU&apikey=' + app_key
        results = requests.get(request).json()
        print(results)
        
        try:
            categories_results = results['features'][0]['properties']['CompanyMetaData']['Categories']
            names_result = results['features'][0]['properties']['CompanyMetaData']['name']
            
            names_short[i + step + offset] = names_result
            
            for category_index in range(len(categories_results)):
                if category_index > 2:
                    break
                
                categories[i + step + offset, category_index] = categories_results[category_index]['name']
                categories_mcc_string[i + step + offset, category_index] = categories_results[category_index]['class']
        except:
            continue
    
    step += 500

In [ ]:
data['org_name'] = names_short
data['category1'] = categories[:, 0]
data['category2'] = categories[:, 1]
data['category3'] = categories[:, 2]
data['category1_mcc_string'] = categories_mcc_string[:, 0]
data['category2_mcc_string'] = categories_mcc_string[:, 1]
data['category3_mcc_string'] = categories_mcc_string[:, 2]
data.to_csv("CustFlowDataset_full_red_mcc.csv", index=None)

In [ ]:
data = data[data['category1_mcc_string'].values != "none"]
data.to_csv("CustFlowDataset_full_red_mcc.csv", index=None)